# Week 4: Basics of Sediment Transport

## Introduction

Why do some rivers carry large cobbles, and others only transport fine silt and clay? In your introductory geology classes, you've probably learned to classify fluvial environments in terms of their <i>energy.</i> For example, you might say that a braided stream is a 'high energy' environment, and a meandering river is 'low energy.' This isn't wrong, but it also isn't very quantitiative. How can we discriminate between 'high energy' and 'low energy' systems? Is a single river always 'high' or 'low' energy, or does the energy of a single river change throughout wet and dry seasons? In this lab, we'll learn how to quantitatively determine sediment transport, and we'll use some new programming tools to help us perform these calculations more efficiently. 

## Learning goals

By the end of this lab, students should be able to:

- Discuss the governing factors of sediment transport 
- Perform calculations to find bed shear stress, and turn those calclations into estimates of maximum transportable grain size
- Write a function to modularize a sediment transport calculation

## Prerequisites

Students participating in this lab should already have basic knowledge of:

- concepts and vocabulary in sedimentary geology
- matplotlib plotting
- numpy arrays

## Background

### Boundary shear stress

When water flows across a stream bed, the water is exerting a <i>force</i> on that surface. Recalling that a stress is a force per unit area, we can also say that the water is exerting a <i>stress</i> on a given area of the stream bed. Stress is a vector quantity, meaning that it has directionality; any stress can be broken down into a normal stress, which acts perpendicular to a surface, and shear stress, which acts parallel to a surface. For problems concerning sediment transport in rivers, we're intersted in the component of stress that acts parallel to the riverbed, exactly at the water-sediment interface. This has a special name: <i><b>boundary shear stress</b></i>. Boundary shear stress, $\tau_b$, is given by the equation

$$\large{\tau_b = \rho g H \sin(\theta)}$$

where $\rho$ is the fluid density ($kg/m^3$), $g$ is the acceleration due to gravity ($m/s^2$), $H$ is the depth of the flow ($m$), and $\theta$ is the slope angle of the riverbed (degrees). From this equation, we can easily see that the boundary shear stress can be increased either by increasing the depth of a flow, or by increasing the channel gradient. Great! Is that all? 

Not quite. This equation alone can't tell us about the size of sediment that a river is capable of moving. To fully understand this, we need the meat and potatoes of sediment transport: the <b><i>critical shear stress</i></b>. But first, we'll quickly recap forces balances as a framework for understanding what exactly the critical shear stress represents.

### A balance of forces

Many problems in physics start with an assumption that a system is at steady state, meaning there is no change happening through time, and these problems model that "steady state" through a force balance. An example of this would be a box sitting on an inclined ramp, but not sliding down the ramp. What's happening? In this example, a physicist would draw us a force balance diagram that shows that the down-ramp driving component of force ($mg\sin\theta$) is perfectly equal and opposite to the up-ramp opposing force of friction ($f$). Thus, the forces are balanced and no motion happens. 

<img src = 'data/force balance.png' align = 'right'> 

But wait! This can't be the case for sediment transport, because the whole point is that <i>motion</i> is occurring! Clearly, while sediment is being transported, the driving forces must outweigh the opposing forces. We won't get into exactly what the "driving" and "opposing" forces are during today's lab; instead, we'll focus on the moment of <i>transport initiation</i>. 

Sediment transport will occur the moment boundary shear stress exceeds the critical shear stress by even the tiniest, most infintesimal amount. This means we can approximate the moment of transport initiation as the moment when boundary shear stress equals critical shear stress.

So, what's critical shear stress?

### Critical shear stress

Critical shear stress, $\tau_c$, is difficult to measure, and for this reason many different empirical formulations exist. Today we'll work with $\tau_c$ as a dimensional number with units $kg/ms^2$, but you should also be aware that both boudary shear stress and critical shear stress and frequently transformed into non-dimensional (unitless) numbers. These are called the Shields stress and critical Shields stress, respectively, and you're very likely to encounter them in the sediment transport literature.

The critical shear stress formulation we will use today comes from Knighton, 1998:

$$\large\tau_c = \tau_c^* g(\rho_p - \rho_f)d_i$$

where $g$ is acceleration due to gravity ($m/s^2$), $\rho_p$ is the particle density ($kg/m^3$), $\rho_f$ is the fluid density ($kg/m^3$), and $d_i$ is the grain size of interest ($m$). Here $\tau_c^*$ is the dimensionless critical shear stress (critical Shields stress) which can be calculated from the following formulation from Andrews, 1983:

$$\large\tau_c^* = 0.0834\left( \frac {d_i}{d_{50}}\right)^{-0.872} $$

where $d_{50}$ is the median grain size, and the exponents and coefficients have been determined empirically.

### A quick note on grain size scales

Grain diameters are reported in meters ($m$) in the calculations shown above, but it is common to report grainsizes in either millimeters ($mm$), or on a dimensionless scale called the <i>phi scale</i>. Working in base 10, grainsize can be converted from millimeters to phi scale ($\phi$) with the following equation:

$$\large\phi = - \frac{log\left(\frac{mm}{\text{ref}}\right)}{log(2)}$$

where $mm$ is grain size in millimeters and $\text{ref}$ is a reference grain size in millimeters, typically chosen to be 1.

Now that we've got the fundamental equations of sediment transport in our toolkit, we're ready to work an example and answer some questions!

## An example

Python allows us to perform numerical operations with ease, but typing out equations in code can be tedious. Instead of typing our boundary shear stress equation over and over, we'll write a function that packages the mathematical operations needed to perform this calculation. You'll quickly see why this is useful. Then, we'll test out our function and plot the outputs.

Steps:
1. Write function using the syntax `def function_name(variables):` and including code documentation
2. Call function for different flow depths
3. Make a plot of boundary shear stress vs flow depth

In [ ]:
# we'll be doing some math and plotting
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# here we'll write our function
# if a variable is given a value when the function is defined, that value will act as a default
# here, density will always be 1000 (kg/m^3) unless the user specifies otherwise
# the red code in triple quotation marks is documentation. It tells us about how this function works.

def calc_boundary_shear(depth, theta, density = 1000, g = 9.81):
    """This function calculates boundary shear stress
    
    Parameters
    ----------
    depth : float
        depth of streamflow in meters
        
    theta : float
        slope of riverbed in degrees
        
    density : float, optional
        density of fluid in kg/m^3
        default is 1000
        
    g : float, optional
        acceleration due to gravity in m/s^2
        default is 9.81
        
    returns boundary shear stress in N/m^2 (kg/m s^2)
        """
    tau_b = density*g*depth*np.sin(np.deg2rad(theta))
    
    return tau_b

In [ ]:
# now let's test our function on a 0.1 degree slope
# remember to assign a variable to "capture" the output

low_flow_tau_b = calc_boundary_shear(0.1, 0.1)
mid_flow_tau_b = calc_boundary_shear(0.2, 0.1)
high_flow_tau_b = calc_boundary_shear(0.5, 0.1)

# print our values to check them out
print('boundary shear stress at low flow: ', low_flow_tau_b, 'kg/ms^2')
print('boundary shear stress at mid flow: ', mid_flow_tau_b, 'kg/ms^2')
print('boundary shear stress at high flow: ', high_flow_tau_b, 'kg/ms^2')

In [ ]:
# finally, let's make a plot to inspect that relationship between boundary shear stress and water depth

# convert our values into arrays
tau_b_values = np.array([low_flow_tau_b, mid_flow_tau_b, high_flow_tau_b])
water_depths = np.array([0.25, 0.5, 1.0])

# make the plot
plt.plot(water_depths, tau_b_values)
plt.title('Change in boundary shear stress with water depth')
plt.xlabel('water depth (m)')
plt.ylabel('boundary shear stress ($\u03C4$) ($kg/ms^2$)')
plt.show()

Fantastic! You've now seen how to write a function and capture the output. You'll use this knowledge to analyze sediment transport in the problems below.

## Questions

1. Write a function that calculates the critical shear stress. Remember that we first have to calculate $\tau_c^*$ before we can find $\tau_c$. Your function should include documentation modeled after the example. 

2. Use your function to find the critical shear stress required to move a quartz grain of the following sizes over a bed with a median grain size equivalent to very coarse sand (you'll have to look up the values of these grain sizes and, if they're given in $\phi$ or $mm$, do some conversions). 

    - coarse pebble
    - very fine sand
    - medium silt

    Which of these particles could be transported at low flow in the stream from the example? Mid flow? High flow?

3. Armed with our knowledge of these equations and our critical thinking skills, explain what we can learn by taking a grainsize sample from a riverbed. What does the median grainsize tell us? And what about the outliers? Answer this question in 4-7 sentences. 

## The end!

## Solutions

In [ ]:
# solution to question 1
def calc_crit_shear(d_i, d_50, particle_density = 2670, fluid_density = 1000, g = 9.81):
    """This function calculates the critical shear stress required to move a quartz grain of a given size (d_i) 
    over a streambed of median grain size d_50
    
    Parameters
    ----------
    d_i : float
        grain size of interest in meters
        
    d_50 : float
        median grain size of streambed in meters
        
    particle_density : float, optional
        density of the particle in kg/m^3
        default is 2670
    
    fluid_density : float, optional
        density of fluid in kg/m^3
        default is 1000
        
    g : float, optional
        acceleration due to gravity in m/s^2
        default is 9.81
        
    returns the critical shear stress in N/m^2 (kg/m s^2)
    """
    
    tau_c_dimensionless = 0.0834 * (d_i/d_50)**(-0.872)
    
    tau_c = tau_c_dimensionless*g*(particle_density - fluid_density)*d_i
    
    return tau_c

In [ ]:
# solution to question 2

# I found grain sizes in mm, so first I convert to m
coarse_pebble = 32/1000
v_fine_sand = (1/8)/1000
med_silt = (1/32)/1000
v_coarse_sand = 2/1000 # this is for the median grain size on the bed

# now I run the function once for each particle size
tau_crit_pebble = calc_crit_shear(coarse_pebble, v_coarse_sand)
tau_crit_sand = calc_crit_shear(v_fine_sand, v_coarse_sand)
tau_crit_silt = calc_crit_shear(med_silt, v_coarse_sand)

# print critical shear stress values
print('critical shear stress for coarse pebble: ', tau_crit_pebble, 'kg/ms^2')
print('critical shear stress for very fine sand: ', tau_crit_sand, 'kg/ms^2')
print('critical shear stress for medium silt: ', tau_crit_silt, 'kg/ms^2')

At low flow, only the medium silt can be transported by our stream. At mid flow, both the sand and silt can be transported. And at the high flow condition, all three of our grain sizes will be transported (plus some!)

3. We can assume that any grain found on a riverbed was transported there by water flowing through the channel. Since $\tau_b = \tau_c$ at the moment of transport initiation, we can say that every grain represents a minimum boundary shear stress that was achieved by water flowing along the streambed at some point in the river's history (and if these grains are still exposed on the surface, that shear stress probably occured in relatively recent history). Through grain size analysis, we could find a median grain size. This would tell us about a rough average of shear stress that the river is exerting on its bed. We could then plug this value into our boundary shear stress equation and, provided we had field data to measure the local gradient, back out an average flow depth for a given stream. Similarly, analyzing the largest grain sizes can tell us about the magnitude of flood events that a particular channel has seen.

## References

Andrews, E.D., Entrainment of Gravel From Naturally Sorted Riverbed Material. Geological Society of
America Bulletin, v.94, p. 1225-1231, October 1983.

Image Retrieved from https://study.com

Knighton, D., Fluvial Forms and Processes A New Perspective. 1998. Oxford University Press Inc.
New York.

